In [1]:
from pyspark import SparkContext

In [2]:
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext(master='local', appName='transformacionesyacciones')
spark2 = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/03 20:15:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
rdd = sc.parallelize([1, 2, 3])

In [22]:
type(rdd)

pyspark.rdd.RDD

In [7]:
rdd.collect()

[1, 2, 3]

In [8]:
sc

<SparkContext master=local appName=transformacionesyacciones>

In [12]:
!ls ../curso/files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [9]:
path = '../curso/files/'

In [13]:
equipos_olimpicos_rdd = sc.textFile(path+'paises.csv').map(lambda x: x.split(','))

In [11]:
equipos_olimpicos_rdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [42]:
equipos_olimpicos_rdd.map(lambda x:( x[2],)).distinct().count()

231

In [12]:
equipos_olimpicos_rdd.map(lambda x:(x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [47]:
equipos_olimpicos_rdd.filter(lambda r: "ARG" in r).collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [48]:
equipos_olimpicos_rdd.count()

1185

In [53]:
equipos_olimpicos_rdd.countApprox(20)

1185

In [14]:
deportista_olimpicos_rdd = sc.textFile(path+'deportista.csv').map(lambda x: x.split(','))
deportista_olimpicos_rdd2 = sc.textFile(path+'deportista.csv').map(lambda x: x.split(','))
deportista_olimpicos_rdd = deportista_olimpicos_rdd.union(deportista_olimpicos_rdd2)

In [67]:
deportista_olimpicos_rdd.top(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['9999', 'Ferenc Bene', '1', '19', '170', '70', '477'],
 ['9999', 'Ferenc Bene', '1', '19', '170', '70', '477'],
 ['9998', 'Kurt Bendlin', '1', '25', '178', '91', '1150']]

In [68]:
equipos_olimpicos_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [66]:
deportista_olimpicos_rdd.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [15]:
deportista_pais = deportista_olimpicos_rdd.map(lambda l: (l[-1], l[:-1])) \
    .join(equipos_olimpicos_rdd.map(lambda l: (l[0], l[2]))) 

In [16]:
deportista_olimpicos_rdd.map(lambda l: (l[-1], l[:-1])) \
    .join(equipos_olimpicos_rdd.map(lambda l: (l[0], l[2]))) \
    .takeSample(False, 6, 25)

[('982',
  (['3045', 'Jos Luis Alonso Berbegal', '1', '37', '165', '76'], 'ESP')),
 ('66', (['51414', 'George Ronald Hyde', '1', '23', '0', '0'], 'AUS')),
 ('702', (['16457', 'Tejbir Bura', '1', '0', '0', '0'], 'NEP')),
 ('970', (['60205', 'Kim SangSik', '1', '23', '184', '70'], 'KOR')),
 ('825',
  (['8120', 'Isaac Barrientos Flores', '1', '18', '190', '108'], 'PUR')),
 ('482', (['40590', 'Gudmunur Gslason', '1', '19', '178', '68'], 'ISL'))]

In [17]:
resultado_ganador = sc.textFile(path+'resultados.csv').map(lambda x: x.split(',')) \
    .filter(lambda l: 'NA' not in l[1])

In [83]:
resultado_ganador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [86]:
deportista_olimpicos_rdd.map(lambda l: (l[-1], l[:-1])) \
    .join(equipos_olimpicos_rdd.map(lambda l: (l[0], l[2]))).take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [88]:
deportista_pais.join(resultado_ganador).take(5)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold'))]

In [92]:
deportista_pais.join(resultado_ganador.map(lambda l: [l[2], l])).take(5)

[('716',
  ((['553', 'John Charles Abrams', '1', '22', '183', '0'], 'NZL'),
   ['1277', 'Gold', '716', '47', '3'])),
 ('716',
  ((['698', 'Paul Douglas Ackerley', '1', '27', '179', '68'], 'NZL'),
   ['1277', 'Gold', '716', '47', '3'])),
 ('716',
  ((['754', 'Craig Robert Adair', '1', '21', '187', '81'], 'NZL'),
   ['1277', 'Gold', '716', '47', '3'])),
 ('716',
  ((['846', 'Valerie Kasanita AdamsVili Price ', '2', '19', '193', '120'],
    'NZL'),
   ['1277', 'Gold', '716', '47', '3'])),
 ('716',
  ((['1511', 'Michael Christopher Aish', '1', '24', '175', '60'], 'NZL'),
   ['1277', 'Gold', '716', '47', '3']))]

In [96]:
deportista_pais.join(resultado_ganador.map(lambda l: [l[4], l])).filter(lambda l: l[0] == '1').take(5)

[('1',
  ((['34666', 'Harald Fereberger', '1', '23', '167', '70'], 'AUT'),
   ['610', 'Gold', '351', '43', '1'])),
 ('1',
  ((['34666', 'Harald Fereberger', '1', '23', '167', '70'], 'AUT'),
   ['1030', 'Bronze', '576', '51', '1'])),
 ('1',
  ((['34666', 'Harald Fereberger', '1', '23', '167', '70'], 'AUT'),
   ['1294', 'Bronze', '727', '19', '1'])),
 ('1',
  ((['34666', 'Harald Fereberger', '1', '23', '167', '70'], 'AUT'),
   ['1808', 'Bronze', '1000', '43', '1'])),
 ('1',
  ((['34666', 'Harald Fereberger', '1', '23', '167', '70'], 'AUT'),
   ['2749', 'Silver', '1530', '15', '1']))]

In [30]:
spark2.stop()

In [5]:

    StructField('juego_id', IntegerType(), False),
    StructField('anio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
 = {
    'Gold': 7,
    'Silver': 5,
    'Bronze': 4
}

In [18]:
paises_medallas = deportista_pais.join(resultado_ganador)

In [21]:
paises_medallas.takeSample(True, 5)

[('591', ((['16317', 'Riardas Bukys', '1', '24', '0', '0'], 'LTU'), 'Silver')),
 ('678',
  ((['19558', 'Chagnaadorjiin Ganzorig', '1', '26', '182', '84'], 'MGL'),
   'Silver')),
 ('656',
  ((['718', 'Luis Angel Acosta', '1', '20', '180', '73'], 'MEX'), 'Bronze')),
 ('656',
  ((['48077', 'Rodolfo Hernndez Vzquez', '1', '26', '172', '80'], 'MEX'),
   'Bronze')),
 ('174', ((['16976', 'Jacob Butula', '1', '21', '0', '0'], 'CAN'), 'Gold'))]

In [25]:
paises_medallas = paises_medallas.map(lambda l: (l[1][0][-1], valores_medallas[l[1][1]]))

In [26]:
from operator import add

In [27]:
result = paises_medallas.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)

In [29]:
result.take(10)

[('CAN', 33142),
 ('ARG', 12908),
 ('HUN', 11272),
 ('MEX', 6056),
 ('RSA', 3480),
 ('BLR', 3080),
 ('MGL', 1530),
 ('USA', 1370),
 ('LTU', 1220),
 ('AZE', 1176)]